# How to Compute Forces

There are 2 approaches.

1. Particle Approach: where we just sum the forces from all the other particles. This is an inefficient algorithm as this scales by $N^2$, where $N$ is the number of particles. A better algorithm may do this via a tree code, which scales as $NlogN$
2. Grid Approach: This solves for the potential at all grid points, and then differentiates to solve for the force. To do this in this case, we must solve the Elliptic Poisson PDE ($\nabla^2\phi(r_{i,j,k})=4\pi G\rho(r_{i,j,k})$). This scales as $N_{grid}$ (the number of grid points)

## Particle Mesh Method (Grid Point)

NOTE: importantly, the grid points may not ALWAYS line up with where the particles are (i.e. the particles may not lie on a grid point)

1. Calculate mass density on a grid.

So if we know $m_i,x_i$, then we can find the density $\rho$ at each point

2. Solve Poisson's equation ont eh grid to compute $\phi$

3. Differentiate to get the forces on the grid $F$

4. Interpolate forces back to particles.

Let's start with step 1, which is calculating the mass density.

For simplicity lets consider a 1D grid. If we have uniform spacing of 1, so the grid cells are bounded by $\{1,2,3,4,5\}$ so the grid points are $\{1.5,2.5,3.5,4.5\}$. Suppose we have a particle of mass $m$ at position $x$. So our mass at our gridpoint $m(g_k)$ is determined completely by this:

$m(g_k)=mW(d)$ where $W(d)$ is the mass assignment function which is a function of $d=|x_i-g_k|$

to get to density we get: $\rho(g_k)=\frac{m(g_k)}{H}$ where $H$ is the grid spacing.

Suppose we have a particle of mass $m$ at position $3.2$

Let's consider a few simple mass assignment functions:

1. Nearest grid point: $W(d)=1$ if $d\leq H\2$ and $0$ otherwise. So our density at that grid point is just the mass. So the mass would be assigned to grid point $3.5$

2. Cloud in cell method: $W(d)=1-d/H$ if $d\leq H$ and $0$ otherwise. So if our mass is at position $3.2$ then it will be assigned proportionally to the 2 cells that it is closest to. So $0.7m$ gets assigned to $3.5$ and $0.3m$ gets assigned to $2.5$. In 2D we share the mass between 4 cells, and in 4D we share the mass between 8 cells.

3. Triangular Shaped Cloud: So now $W(d)=\frac{3}{4}-(\frac{d}{H})^2$ for $d\leq H/2$ etc. etc. Look to lectures for more details, but essentially we're just spreading out the particle mass by taking a triangular shaped cloud. 

In 3D, we just have the mass assignment function as just the product of the 1D mass assignment functions in each dimension.

Now to solve Poisson's equation (this is step 2). There are 2 methods to solve it.

1. Solve by Fourier Transform. 

We know that the solution is

$phi(x,y,z)=-\int \frac{G\rho (x',y',z')}{|\vec(x)-\vec(x)'|}dx'dy'dz'$

We will use the convolution theorem to solve this integral. A reminder that the convolution of $g$ and $h$ is:

$g*h=\int_{\infty}^{\infty}g(x')h(x-x')dx'$

So the fourier transform $g*h$ is just the product of the fourier transforms of $g$ and $h$.

So if we pick $g=\rho$ and $h=\frac{-G}{|\vec(x)-\vec(x)'|}$ ($h$ is the green's function btw)

so now we get:

$\hat{\phi}(k)=\hat{\rho}(k)\hat{h}(k)$

so we only have to fourier transform 2 functions and multiply per time step INSTEAD of solving an integral each time step. 

Furthermore, the fourier transform of the greens function can be done just once.

So the (sub)steps for this are:

1. Fast Fourier Transform (FFT) the Density Field to get $\hat{\rho}$

2. Multiply with the greens function

3. FFT backward to get $\phi$

however, FFT requires a uniform grid. so if we get a uniform grid, this scales as the number of grid points!

Onto differentiation! We can do this via simple finite differencing. So to get the $x$ component of the force $k,l,m$ we can use the centred difference scheme to get a 2nd order accurate answer!

These are computed at every grid point, but remember that the particles might not actually sit on a grid point. So we must interpolate

Onto the final step! This is to interpolate the forces on the grid points onto the forces at the positions of the particles. So we get this:

$F(x_i)=\sum W(x_i-g_{k,l,m})F_{k,l,m}$

IMPORTANTLY, we must use the same $W$ here as the one we used in mass assignment in order to ensure that 2 particles experience equal and opposite forces to conserve momentum

Now the 2nd method for solving for the potential is a 'Relaxation Solver'

The good advantage is that one is NOT limited to a uniform grid. 

The idea is that one can change the Elliptic PDE into a pseudo-time dependent problem (parabolic PDE).

so let's change the Poisson eqn to this

$\frac{\partial \phi}{\partial \tau}=\nabla ^2 \phi - 4\pi G \rho$

So $\tau$ is our pseudo time. So we solve this solution iteratively until $\frac{\partial \phi}{\partial \tau}=0$, in which case it reduces to the original PDE.

The nice thing is that it looks like a diffusion eqn!

So in 2D (easily generalisable to 3D):

$\frac{\phi^{n+1}_{j,k}-\phi^n_{j,k}}{\delta \tau}=\frac{\phi^{n}_{j+1,k}-2\phi^{n}_{j,k}+\phi^{n}_{j-1,k}}{\delta x^2}+\frac{\phi^{n}_{j,k+1}-2\phi^{n}_{j,k}+\phi^{n}_{j,k-1}}{\delta y^2}-4\pi G \rho_{j,k}$

This can be relatively easily rearranged, and I will not bother typing it out in Latex. Please check the lectures at time stamp 44:50 for this.

So what should we take for our $\delta \tau$? Recall that we had a limitation on our timestep in order for the scheme to be stable! (Note that this scheme is explicit).

In 2D the limitation is:

$\frac{\delta \tau}{\delta x^2}+\frac{\delta \tau}{\delta y^2} \leq \frac{1}{2}$

So $\frac{\delta \tau}{\delta x^2} \leq \frac{1}{4}$ if $\delta x = \delta y$

so if we take the maximum, we get a very simple scheme (at 49:12) in the lectures.

This method is called the __Jacobi Method__. 

Recall that this is an elliptical PDE, so we are solving a __boundary value__ problem. So the potential will be known at the boundary and one must solve the interior points simultaneously. In real life examples, internal grid points will number ~1000s or ~millions, so it will be incredibly costly to invert a matrix every time. So most of the time the iterative method is much better!



on the first iteration, we still do not try to solve simultaneously. Instead, we guess the unknown values to get started on the first grid, and then we just keep iterating through each grid point!

Then after we compute all the internal grid points we check if $max|\phi^{n+1}_{k,j}-\phi^n_{k,j}|<\epsilon$ where $\epsilon$ is some number, thus ensuring that the partial derivative of phi wrt tau is 0 so then we get the solution to the elliptic pde!

Turns out that hte JAcobi method converges very slowly and IS VERY sensitve to the inital guess. LEt's look at a better method

The Jacobi Method in 2D is:

$\Phi^{n+1}_{i,j}=\frac{1}{4}(\Phi^n_{j+1,k}+\Phi^n_{j-1,k}+\Phi^n_{j,k+1}+\Phi^n_{j,k-1}-4\pi G \rho _{i,j} (\delta x)^2)$

The Gauss Seidal Method is exactly the same as the Jacobi method at the very first step, but instead at the next step we use the previous calculated result of $\Phi^{n+1}$ to replace one of the unknowns of $\Phi^n$

Both methods eliminate short wavelength errors (i.e. errors comparable to the grid spacing), but inefficient at eliminating large wavelength errors. This is because a grid point only interacts with its neighbours. A solution to this is to use __multi-grid relaxation__ techniques. 

this works via overlaying multiple (coarser) grids on top of each other with different grid spacing. We try to get a solution that converges for coarser grids, and then we use that solution as an initial guess for iterating on the finer grid. 

The way you sweep through the internal grid points are also not that important

So what is the force resolution for these grid methods?

So ideally, for a $F \sim \frac{1}{r^2}$, then we should get a constant straight negative slope when we plot $lnF$ against $lnr$.

Unsurprisingly, our method follows this quite nicely until we reach a distance lower than our minimum grid spacing. To get more resolution, we just need finer grids.